In [ ]:
import importlib
import pandas as pd
import random
import glob
import matplotlib.pyplot as plt
import numpy as np
from tqdm.notebook import tqdm
import sys
path_utils = r'C:\Users\gianluca.nogara\Desktop\Repo\Vaccines_Discussion_Italy\tweet_utils'
sys.path.append(path_utils)
import utils
path_files = r"C:\Users\gianluca.nogara\Desktop\Repo\Vaccines_Discussion_Italy\Italian\Files"

# Obtain new Data

In [ ]:
tweets = pd.read_csv(path_files + r"\Tweets\tweets.csv", lineterminator="\n", encoding="utf-8", low_memory=False)

In [ ]:
toxic = pd.read_csv(path_files + r"\tweets\tweets_toxicity.csv", lineterminator="\n",low_memory=False, encoding="utf-8", usecols=["id", "toxicity"])
sentiment = pd.read_csv(r"C:\Users\gianluca.nogara\Desktop\Repo\Vaccines_Discussion_Italy\Italian\script_directory_output\sentiment\merged.csv", lineterminator="\n", low_memory=False, encoding="utf-8", usecols=["id", "sentiment", "emotion"])
in_deg = pd.read_csv(path_files + r"\networks\in_degree.csv")
in_deg.rename(columns={"user": "user_screen_name", "degree": "in_degree"}, inplace=True)
deg = pd.read_csv(path_files + r"\networks\degree.csv")
deg.rename(columns={"user": "user_screen_name"}, inplace=True)
community = pd.read_csv(path_files + r"\networks\louvain.csv")
community.rename(columns={"name": "user_screen_name"}, inplace=True)
domains = pd.read_csv(path_files + r"\domains\result_cred_score_thresh_10_low_domains.csv", low_memory=False, encoding="utf-8", lineterminator="\n", usecols=["user_screen_name", "credibility_score"])
deg_cen = pd.read_csv(path_files + r"\networks\user_deg_centrality.csv", low_memory=False, encoding="utf-8", lineterminator="\n")
deg_cen.rename(columns={"name": "user_screen_name"}, inplace=True)
ev_cen = pd.read_csv(path_files + r"\networks\user_ev_centrality.csv", low_memory=False, encoding="utf-8", lineterminator="\n")
ev_cen.rename(columns={"name": "user_screen_name"}, inplace=True)
tweets = tweets.merge(toxic, on="id", how="left")
tweets = tweets.merge(sentiment, on="id", how="left")
tweets = tweets.merge(in_deg, on="user_screen_name", how="left")
tweets = tweets.merge(deg, on="user_screen_name", how="left")
tweets = tweets.merge(community, on="user_screen_name", how="left")
tweets = tweets.merge(domains, on="user_screen_name", how="left")
tweets = tweets.merge(deg_cen, on="user_screen_name", how="left")
tweets = tweets.merge(ev_cen, on="user_screen_name", how="left")

In [ ]:
df_urls = tweets.loc[tweets['urls'] != '[]']
df_urls = [x.split()[3].translate({ord("'"): None}).replace(",","") for x in df_urls["urls"]]
df_urls = [x if ("https://youtu.be" in x) or ("https://www.youtube.com" in x) else np.nan for x in df_urls]
df_urls = list(pd.Series(df_urls).dropna())
lst = []
for i in df_urls:
    if 'playlist' in i:
        lst.append(i)
for i in df_urls:
    if 'www.youtube.com/channel' in i:
        lst.append(i)
l3 = [x for x in df_urls if x not in lst]
l3 = list(set(l3))
# df_yt_domains = pd.DataFrame(l3, columns=["URL"]).to_csv(r"C:\Users\gianl\Desktop\Gi\Supsi\Vaccines_Discussion_Italy\Italian\files\domains_yt.csv",
#                                                          line_terminator="\n", index=False, encoding='utf-8')

# Read data

In [ ]:
path_base = r"C:\Users\gianluca.nogara\Desktop\Repo\Vaccines_Discussion_Italy\Italian"
suspended_files = glob.glob(path_base + r"\script_directory_output\suspended_users\out*.csv")
df = pd.DataFrame()
for file in suspended_files:
    temp = pd.read_csv(file, lineterminator="\n", encoding="utf-8", low_memory=False)
    df = df.append(temp)
df.drop_duplicates(subset=["name"], inplace=True)
df.reset_index(drop=True, inplace=True)
lst_suspended = df[df["status"].str.contains("'code': 63")]

In [ ]:
novax = pd.read_csv(path_files + r"\users\bad.csv", lineterminator="\n", low_memory=False, encoding="utf-8")
novax = list(novax["screen_name"])
users = pd.read_csv(path_files + r"\users\other_users.csv", lineterminator="\n", low_memory=False, encoding="utf-8")
users = list(users["name"])
provax = pd.read_csv(path_files + r"\users\good.csv", lineterminator="\n", low_memory=False, encoding="utf-8")
provax = list(provax["name"])
suspended = list(lst_suspended["name"])
novax_activities = tweets[tweets["user_screen_name"].isin(novax)]
users_activities = tweets[tweets["user_screen_name"].isin(users)]
provax_activities = tweets[tweets["user_screen_name"].isin(provax)]
suspended_activities =  tweets[tweets["user_screen_name"].isin(suspended)]
novax_rt_in = tweets[tweets["rt_user_screen_name"].isin(novax)]
users_rt_in = tweets[tweets["rt_user_screen_name"].isin(users)]
provax_rt_in = tweets[tweets["rt_user_screen_name"].isin(provax)]
suspended_rt_in =  tweets[tweets["rt_user_screen_name"].isin(suspended)]

In [ ]:
print(f"Novax activities: {len(novax_activities)}")
print(f"Users activities: {len(users_activities)}")
print(f"Provax activities: {len(provax_activities)}")
print(f"Suspended activities: {len(suspended_activities)}")
print("__________________________________________")
print(" ")
print(f"Novax rt received: {len(novax_rt_in)}")
print(f"Users rt received: {len(users_rt_in)}")
print(f"Provax rt received: {len(provax_rt_in)}")
print(f"Suspended rt received: {len(suspended_rt_in)}")

# Activities

In [ ]:
x = utils.split(novax_activities)
labels = list(x.keys())
values = list(x.values())
values = [i/sum(values) for i in values]
plt.bar(labels, values)
plt.title("NOVAX activities")
plt.show()
x = utils.split(users_activities)
labels = list(x.keys())
values = list(x.values())
values = [i / sum(values) for i in values]
plt.bar(labels, values)
plt.title("USERS activities")
plt.show()
x = utils.split(provax_activities)
labels = list(x.keys())
values = list(x.values())
values = [i / sum(values) for i in values]
plt.bar(labels, values)
plt.title("PROVAX activities")
plt.show()
x = utils.split(suspended_activities)
labels = list(x.keys())
values = list(x.values())
values = [i / sum(values) for i in values]
plt.bar(labels, values)
plt.title("SUSPENDED activities")
plt.show()


# Hashtags

In [ ]:
#import importlib
#importlib.reload(utils)

In [ ]:
val = pd.Series(utils.hashtag_extraction(novax_activities)).value_counts().sort_values(ascending=False)
fig = plt.figure()
ax = val[:10].plot(kind='barh', color='green')
ax.set_title('Most HASHTAGS shared by NOVAX')
ax.invert_yaxis()
plt.show()

In [ ]:
val = pd.Series(utils.hashtag_extraction(users_activities)).value_counts().sort_values(ascending=False)
fig = plt.figure()
ax = val[:10].plot(kind='barh', color='lightgreen')
ax.set_title('Most HASHTAGS shared by USERS')
ax.invert_yaxis()
plt.show()

In [ ]:
val = pd.Series(utils.hashtag_extraction(provax_activities)).value_counts().sort_values(ascending=False)
fig = plt.figure()
ax = val[:10].plot(kind='barh', color='orange')
ax.set_title('Most HASHTAGS shared by USERS')
ax.invert_yaxis()
plt.show()

In [ ]:
val = pd.Series(utils.hashtag_extraction(suspended_activities)).value_counts().sort_values(ascending=False)
fig = plt.figure()
ax = val[:10].plot(kind='barh', color='red')
ax.set_title('Most HASHTAGS shared by SUSPENDED')
ax.invert_yaxis()
plt.show()

# Urls

In [ ]:
x = utils.extract_domain_list(novax_activities)
novax_urls = []
lst_not_plot = ["twitter.com", "bit.ly", "ow.ly"]
cont = 0
for i in x:
    if i != []:
        for j in i:
            if (j not in lst_not_plot):
                if j == "youtu.be":
                    j = "youtube.com"
                novax_urls.append(j)
val = pd.Series(novax_urls).value_counts().sort_values(ascending=False)
fig = plt.figure()
ax = val[:15].plot(kind='barh', color='green')
ax.set_title('Most URLS shared by NOVAX')
ax.invert_yaxis()
plt.show()

In [ ]:
x = utils.extract_domain_list(users_activities)
users_urls = []
lst_not_plot = ["dlvr.it", "twitter.com", "tinyurl.com", "trib.al", "bit.ly", "ow.ly"]
cont = 0
for i in x:
    if i != []:
        for j in i:
            if j not in lst_not_plot:
                if j == "youtu.be":
                    j = "youtube.com"
                users_urls.append(j)
val = pd.Series(users_urls).value_counts().sort_values(ascending=False)
fig = plt.figure()
ax = val[:15].plot(kind='barh', color='lightgreen')
ax.set_title('Most URLS shared by USERS')
ax.invert_yaxis()
plt.show()

In [ ]:
x = utils.extract_domain_list(provax_activities)
users_urls = []
lst_not_plot = ["dlvr.it", "twitter.com", "tinyurl.com", "trib.al", "bit.ly", "ow.ly"]
cont = 0
for i in x:
    if i != []:
        for j in i:
            if j not in lst_not_plot:
                if j == "youtu.be":
                    j = "youtube.com"
                users_urls.append(j)
val = pd.Series(users_urls).value_counts().sort_values(ascending=False)
fig = plt.figure()
ax = val[:15].plot(kind='barh', color='orange')
ax.set_title('Most URLS shared by PROVAX')
ax.invert_yaxis()
plt.show()

In [ ]:
x = utils.extract_domain_list(suspended_activities)
users_urls = []
lst_not_plot = ["dlvr.it", "twitter.com", "tinyurl.com", "trib.al", "bit.ly", "ow.ly"]
cont = 0
for i in x:
    if i != []:
        for j in i:
            if j not in lst_not_plot:
                if j == "youtu.be":
                    j = "youtube.com"
                users_urls.append(j)
val = pd.Series(users_urls).value_counts().sort_values(ascending=False)
fig = plt.figure()
ax = val[:15].plot(kind='barh', color='red')
ax.set_title('Most URLS shared by SUSPENDED')
ax.invert_yaxis()
plt.show()

# YouTube

In [ ]:
def extract_youtube(df):
    value = []
    for i in (df.itertuples()):
        url_exp = i.urls.split(" ")
        lst_inside = []
        for exp in range(len(url_exp)):
            if url_exp[exp] == "'expanded_url':":
                lst_inside.append((url_exp[exp+1][1:-2], i.id))
        value.append(lst_inside)
    urls = []
    ids = []
    for i in value:
        for j in i:
            if "youtu" in j[0]:
                urls.append(j[0])
                ids.append(j[1])
    df_domains_ids = pd.DataFrame(list(zip(ids, urls)), columns=['id', 'url'])
    df_domains_ids.drop_duplicates(subset=["id"], keep="last", inplace=True)
    return df_domains_ids

In [ ]:
novax_yt = extract_youtube(novax_activities)
users_yt = extract_youtube(users_activities)
provax_yt = extract_youtube(provax_activities)
suspended_yt = extract_youtube(suspended_activities)

In [ ]:
yt_info = pd.read_csv(r"C:\Users\gianluca.nogara\Desktop\Repo\Vaccines_Discussion_Italy\Italian\script_directory_output\youtube\youtube_info.csv",
                      lineterminator="\n", low_memory=False, encoding="utf-8")
yt_info.drop_duplicates(subset=["url"], keep="last", inplace=True)

In [ ]:
novax_yt = yt_info.merge(novax_yt, on="url", how='left')
novax_yt = novax_yt[novax_yt['id'].notna()]
users_yt = yt_info.merge(users_yt, on="url", how='left')
users_yt = users_yt[users_yt['id'].notna()]
provax_yt = yt_info.merge(provax_yt, on="url", how='left')
provax_yt = provax_yt[provax_yt['id'].notna()]
suspended_yt = yt_info.merge(suspended_yt, on="url", how='left')
suspended_yt = suspended_yt[suspended_yt['id'].notna()]

In [ ]:
print(f"NOVAX YouTube video availability: {round(len(novax_yt[novax_yt['available']==True])/len(novax_yt),3)*100}%")
print(f"USERS YouTube video availability: {round(len(users_yt[users_yt['available']==True])/len(users_yt),3)*100}%")
print(f"PROVAX YouTube video availability: {round(len(provax_yt[provax_yt['available']==True])/len(provax_yt),3)*100}%")
print(f"SUSPENDED YouTube video availability: {round(len(suspended_yt[suspended_yt['available']==True])/len(suspended_yt),3)*100}%")
# pd.Series(available["channel"]).value_counts()

# Most active, OUT RTs and IN RTs

In [ ]:
val = pd.Series(novax_activities["user_screen_name"]).value_counts().sort_values(ascending=False)
fig = plt.figure()
ax = val[:15].plot(kind='barh', color='green')
ax.set_title('Most ACTIVE NOVAX')
ax.invert_yaxis()
plt.show()
val = pd.Series(users_activities["user_screen_name"]).value_counts().sort_values(ascending=False)
fig2 = plt.figure()
ax2 = val[:15].plot(kind='barh', color='lightgreen')
ax2.set_title('Most ACTIVE USERS')
ax2.invert_yaxis()
plt.show()
val = pd.Series(provax_activities["user_screen_name"]).value_counts().sort_values(ascending=False)
fig3 = plt.figure()
ax3 = val[:15].plot(kind='barh', color='orange')
ax3.set_title('Most ACTIVE PROVAX')
ax3.invert_yaxis()
plt.show()
val = pd.Series(suspended_activities["user_screen_name"]).value_counts().sort_values(ascending=False)
fig4 = plt.figure()
ax4 = val[:15].plot(kind='barh', color='red')
ax4.set_title('Most ACTIVE SUSPENDED')
ax4.invert_yaxis()
plt.show()

In [ ]:
val = pd.Series(novax_activities["rt_user_screen_name"]).value_counts().sort_values(ascending=False)
fig = plt.figure()
ax = val[:15].plot(kind='barh', color='green')
ax.set_title('Most OUT RTs of NOVAX')
ax.invert_yaxis()
plt.show()
val = pd.Series(users_activities["rt_user_screen_name"]).value_counts().sort_values(ascending=False)
fig2 = plt.figure()
ax2 = val[:15].plot(kind='barh', color='lightgreen')
ax2.set_title('Most OUT RTs of USERS')
ax2.invert_yaxis()
plt.show()
val = pd.Series(provax_activities["rt_user_screen_name"]).value_counts().sort_values(ascending=False)
fig3 = plt.figure()
ax3 = val[:15].plot(kind='barh', color='orange')
ax3.set_title('Most OUT RTs of PROVAX')
ax3.invert_yaxis()
plt.show()
val = pd.Series(suspended_activities["rt_user_screen_name"]).value_counts().sort_values(ascending=False)
fig4 = plt.figure()
ax4 = val[:15].plot(kind='barh', color='red')
ax4.set_title('Most OUT RTs of SUSPENDED')
ax4 .invert_yaxis()
plt.show()

In [ ]:
val = pd.Series(novax_rt_in["user_screen_name"]).value_counts().sort_values(ascending=False)
fig = plt.figure()
ax = val[:15].plot(kind='barh', color='green')
ax.set_title('Most IN RTs of NOVAX')
ax.invert_yaxis()
plt.show()
val = pd.Series(users_rt_in["user_screen_name"]).value_counts().sort_values(ascending=False)
fig2 = plt.figure()
ax2 = val[:15].plot(kind='barh', color='lightgreen')
ax2.set_title('Most IN RTs of USERS')
ax2.invert_yaxis()
plt.show()
val = pd.Series(provax_rt_in["user_screen_name"]).value_counts().sort_values(ascending=False)
fig3 = plt.figure()
ax3 = val[:15].plot(kind='barh', color='orange')
ax3.set_title('Most IN RTs of PROVAX')
ax3.invert_yaxis()
plt.show()
val = pd.Series(suspended_rt_in["user_screen_name"]).value_counts().sort_values(ascending=False)
fig4 = plt.figure()
ax4 = val[:15].plot(kind='barh', color='red')
ax4.set_title('Most IN RTs of SUSPENDED')
ax4 .invert_yaxis()
plt.show()

# Credibility score, Centrality and Community

In [ ]:
na = novax_activities[novax_activities["user_screen_name"].notna()]
novax_users = na.drop_duplicates(subset=["user_screen_name"], keep="last")[["user_screen_name", "ev", "deg_cen", "in_degree", "degree", "credibility_score", "community"]]
ua = users_activities[users_activities["user_screen_name"].notna()]
users_users = ua.drop_duplicates(subset=["user_screen_name"], keep="last")[["user_screen_name", "ev", "deg_cen", "in_degree", "degree", "credibility_score", "community"]]
pa = provax_activities[provax_activities["user_screen_name"].notna()]
provax_users = pa.drop_duplicates(subset=["user_screen_name"], keep="last")[["user_screen_name", "ev", "deg_cen", "in_degree", "degree", "credibility_score", "community"]]
sa = suspended_activities[suspended_activities["user_screen_name"].notna()]
suspended_users = sa.drop_duplicates(subset=["user_screen_name"], keep="last")[["user_screen_name", "ev", "deg_cen", "in_degree", "degree", "credibility_score", "community"]]

In [ ]:
print(f"Mean NOVAX credibility value {round(np.nanmean(novax_users['credibility_score'])*100,2)}%")
print(f"Mean USERS credibility value {round(np.nanmean(users_users['credibility_score'])*100,2)}%")
print(f"Mean PROVAX credibility value {round(np.nanmean(provax_users['credibility_score'])*100,2)}%")
print(f"Mean SUSPENDED credibility value {round(np.nanmean(suspended_users['credibility_score'])*100,2)}%")

In [ ]:
val = pd.Series(novax_users["community"]).value_counts().sort_values(ascending=False)
ax = val.plot(kind='barh', color='green')
ax.set_title('Communities of NOVAX')
ax .invert_yaxis()
plt.show()
val = pd.Series(users_users["community"]).value_counts().sort_values(ascending=False)
ax = val.plot(kind='barh', color='lightgreen')
ax.set_title('Communities of USERS')
ax .invert_yaxis()
plt.show()
val = pd.Series(provax_users["community"]).value_counts().sort_values(ascending=False)
ax = val.plot(kind='barh', color='orange')
ax.set_title('Communities of PROVAX')
ax .invert_yaxis()
plt.show()
val = pd.Series(suspended_users["community"]).value_counts().sort_values(ascending=False)
ax = val.iloc[:15].plot(kind='barh', color='red')
ax.set_title('Communities of SUSPENDED')
ax .invert_yaxis()
plt.show()

In [ ]:
print(f"Mean NOVAX ev centrality value {round(np.nanmean(novax_users['ev']),5)}")
print(f"Mean USERS ev centrality value {round(np.nanmean(users_users['ev']),5)}")
print(f"Mean PROVAX ev centrality value {round(np.nanmean(provax_users['ev']),5)}")
print(f"Mean SUSPENDED ev centrality value {round(np.nanmean(suspended_users['ev']),5)}")

# Sentiment and Emotion

In [ ]:
print(f"NOVAX have {round(len(novax_activities[novax_activities['sentiment']=='negative'])/len(novax_activities)*100,2)}% of Negative sentences")
print(f"USERS have {round(len(users_activities[users_activities['sentiment']=='negative'])/len(users_activities)*100,2)}% of Negative sentences")
print(f"PROVAX have {round(len(provax_activities[provax_activities['sentiment']=='negative'])/len(provax_activities)*100,2)}% of Negative sentences")
print(f"SUSPENDED have {round(len(suspended_activities[suspended_activities['sentiment']=='negative'])/len(suspended_activities)*100,2)}% of Negative sentences")

In [ ]:
val = pd.Series(novax_activities["emotion"]).value_counts().sort_values(ascending=False)
ax = val.plot(kind='barh')
ax.set_title('Emotions of NOVAX')
ax .invert_yaxis()
plt.show()
val = pd.Series(users_activities["emotion"]).value_counts().sort_values(ascending=False)
ax = val.plot(kind='barh')
ax.set_title('Emotions of USERS')
ax .invert_yaxis()
plt.show()
val = pd.Series(provax_activities["emotion"]).value_counts().sort_values(ascending=False)
ax = val.plot(kind='barh')
ax.set_title('Emotions of PROVAX')
ax .invert_yaxis()
plt.show()
val = pd.Series(suspended_activities["emotion"]).value_counts().sort_values(ascending=False)
ax = val.plot(kind='barh')
ax.set_title('Emotions of SUSPENDED')
ax .invert_yaxis()
plt.show()

# Toxicity

In [ ]:
print(f"Mean NOVAX toxicity value {round(np.nanmean(novax_activities['toxicity']),2)}")
print(f"Mean USERS toxicity value {round(np.nanmean(users_activities['toxicity']),2)}")
print(f"Mean PROVAX toxicity value {round(np.nanmean(provax_activities['toxicity']),2)}")
print(f"Mean SUSPENDED toxicity value {round(np.nanmean(suspended_activities['toxicity']),2)}")

# Toxicity

In [ ]:
print(f"Mean NOVAX toxicity value {round(np.nanmean(novax_activities['toxicity']),2)}")
print(f"Mean USERS toxicity value {round(np.nanmean(users_activities['toxicity']),2)}")
print(f"Mean PROVAX toxicity value {round(np.nanmean(provax_activities['toxicity']),2)}")
print(f"Mean SUSPENDED toxicity value {round(np.nanmean(suspended_activities['toxicity']),2)}")

# Toxicity

In [160]:
print(f"Mean NOVAX toxicity value {round(np.nanmean(novax_activities['toxicity']),2)}")
print(f"Mean USERS toxicity value {round(np.nanmean(users_activities['toxicity']),2)}")
print(f"Mean PROVAX toxicity value {round(np.nanmean(provax_activities['toxicity']),2)}")
print(f"Mean SUSPENDED toxicity value {round(np.nanmean(suspended_activities['toxicity']),2)}")

Mean NOVAX toxicity value 0.12
Mean USERS toxicity value 0.08
Mean PROVAX toxicity value 0.04
Mean SUSPENDED toxicity value 0.15
